# Projekt - model klasyfikacyjny

Cel projektu: Stworzenie systemu do alertów (progonozowanie czy wypożyczeń będzie więcej niż zwrotów).
Alert powinien dotyczy kolejnej godziny. Tak, aby móc wysłać pracowników w rejony z niedoborem rowerów i przewieźć tam rezerwowe rowery lub pojazdów z innych lokalizacji.

Zadania do realizacji
1. Data preprocessing:
    - Pobranie danych
    - Wstępne sprawdzenie danych.
    - Przekodowanie zmiennej czasowej na datę
    - Przekodowanie zmiennej departure_id oraz return_id na string
    - Stworzenie zmiennych z daty: godzina,miesiąc, kwartał.
    - Stworzenie nowej zmiennej kategorycznej (y): Czy liczba wypożyczeń w bieżącej godzinie jest większa niż liczba zwrotów.
    - Enkoding zmiennej departure id
    - Stworzenie lagów (wartości z poprzednich okresów):
        - wartości dla danej stacji z poprzedniej godziny
        - średnie wartości dla stacji z poprzedniego dnia
        - średnie wartości ogółu z poprzedniego dnia i godziny
    - Selekcja zmiennych
    - Detekcja outlierów.
2. Optymalizacja modelu:
    - Wykorzystanie jednego z poznanych algorytmów optymalizacyjnych.
    - W przypadku niezadowalających wyników, testy na innym algorytmie.

In [ ]:
import pandas as pd

1. Data preprocessing
- Pobranie danych

In [ ]:
# Wczytanie danych - liczba wypożyczeń
df = pd.read_parquet('data/hourly_data_per_station.parquet')


In [ ]:
# Wczytanie danych - liczba zwrotów
df_agg_ret=pd.read_parquet('data/hourly_data_per_station_returns.parquet')

- Wstępne sprawdzenie danych.

In [ ]:
# head
df_agg_ret.head()

In [ ]:
# head
df.head()

In [ ]:
# info
df.info()

In [ ]:
# info
df_agg_ret.info()

- Przekodowanie zmiennej czasowej na datę

In [ ]:
# departure date
df['departure_date_hours'] = pd.to_datetime(df['departure_date_hours'])

In [ ]:
# return date
df_agg_ret['return_date_hours'] = pd.to_datetime(df_agg_ret['return_date_hours'])

  - Stworzenie zmiennych z daty: godzina,miesiąc, kwartał.

In [ ]:
df['hour'] = df['departure_date_hours'].dt.hour
df['month'] = df['departure_date_hours'].dt.month
df['quarter'] = df['departure_date_hours'].dt.quarter

- Stworzenie nowej zmiennej kategorycznej (y): Czy liczba wypożyczeń w bieżącej godzinie jest większa niż liczba zwrotów.

In [ ]:
# polaczenie danych
df_merged = df.merge(df_agg_ret[['return_date_hours','return_id','nr_of_returns']], left_on =[
    'departure_id', 'departure_date_hours'], right_on=['return_id','return_date_hours'], how='left')

In [ ]:
# wielkosci poszczegolnych ramek
df.shape

In [ ]:
df_merged.shape

In [ ]:
# sprawdzenie wartosci pustych
df_merged[df_merged['nr_of_returns'].isna()]

In [ ]:
# imputacja danych
df_merged['nr_of_returns'] = df_merged['nr_of_returns'].fillna(0)

In [ ]:
# zmienna y kategoryczna
df_merged['y_cat'] = (((df_merged['nr_of_departures']-1) > df_merged['nr_of_returns'])).astype(int)

In [ ]:
# udział wartosci y
df_merged['y_cat'].value_counts()/df.shape[0]

 - Enkoding zmiennej departure id

In [ ]:
from sklearn.preprocessing import TargetEncoder

In [ ]:
# obiekt taretencoder
te = TargetEncoder(target_type='continuous').fit(df_merged[['departure_id']],df_merged['nr_of_departures'])

In [ ]:
# dodanie zmiennej enkodowanej
df_merged['departure_id_encoded'] = te.transform(df_merged[['departure_id']])

In [ ]:
# sprawdzenie - head
df_merged.head()

- Stworzenie lagów (wartości z poprzednich okresów):

    - wartości dla danej stacji z poprzedniej godziny
    - średnie wartości dla stacji z poprzedniego dnia
    - średnie wartości ogółu z poprzedniego dnia i godziny

In [ ]:
# pobranie stworzonych funkcji
from help_function import agg_data, lag_n

In [ ]:
# kalkulacja lagów
lag_cols = ['nr_of_departures','nr_of_returns','Air temperature (degC)','distance (m)','duration (sec.)','y_cat']

In [ ]:
# stworzenie daty z dokładnoscia do dnia
df_merged["departure_date"] = df_merged['departure_date_hours'].dt.date
df_merged.head()

In [ ]:
# agregacja danych do dnia
daily_data = agg_data(df_merged,'departure_date',{
    'nr_of_departures': 'sum',
    'Air temperature (degC)': 'mean',
    'distance (m)': 'mean',
    'duration (sec.)': 'mean'
})

In [ ]:
# zmiana nazw kolumn
daily_data.columns = ['yt_' + i for i in daily_data.columns]
daily_data.head()

In [ ]:
# wyznaczenie wczorajszej daty
df_merged["yesterday_date"] = df_merged['departure_date'] - pd.Timedelta(days=1)
df_merged.head()

In [ ]:
# połączenie ramek danych
print(df_merged.shape)
df_merged = df_merged.merge(
    daily_data, left_on = 'yesterday_date', right_on='yt_departure_date', how='left').fillna(0)
print(df_merged.shape)

In [ ]:
# dane dzienne per stacja
daily_data_station = agg_data(df_merged,['departure_date','departure_id'],{
    'nr_of_departures': 'sum',
    'Air temperature (degC)': 'mean',
    'distance (m)': 'mean',
    'duration (sec.)': 'mean'
})

In [ ]:
# zmiana nazw kolumn
daily_data_station.columns = ['yt_station_' + i for i in daily_data_station.columns]
daily_data_station.head()

In [ ]:
# połączenie ramek danych
print(df_merged.shape)
df_merged = df_merged.merge(
    daily_data_station, 
    left_on = ['yesterday_date','departure_id'], 
    right_on=['yt_station_departure_date','yt_station_departure_id'], 
    how='left').fillna(0)
print(df_merged.shape)

In [ ]:
# Stworzenie funkcji prepare data, która przetworzy ramkę danych i doda do niej wymagane zmienne do późniejszej predykcji modelu
def prepare_data(df):
    pass

- Selekcja zmiennych

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector, RFE
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# kolumny w ramce danych
df_merged.columns

In [ ]:
# lista potencjalnych zmiennych
potential_x_names = ['departure_id_encoded','Air temperature (degC)','nr_of_departures_lag_1',
       'nr_of_returns_lag_1', 'Air temperature (degC)_lag_1',
       'distance (m)_lag_1', 'duration (sec.)_lag_1','yt_nr_of_departures',
       'yt_Air temperature (degC)', 'yt_distance (m)', 'yt_duration (sec.)',
       'yt_station_nr_of_departures', 'yt_station_Air temperature (degC)',
       'yt_station_distance (m)', 'yt_station_duration (sec.)'
       ]
len(potential_x_names)

In [ ]:
# selekcja zmiennych
seq = RFE(estimator=DecisionTreeClassifier(max_depth=5),n_features_to_select=10)
seq.fit(df_merged[potential_x_names],df_merged['y_cat'])

In [ ]:
# lista finalnych zmiennych
x_names = seq.get_feature_names_out()
x_names

- Detekcja outlierów.

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
# Definicja obiektu
iso_forest = IsolationForest(bootstrap=True,)

In [ ]:
# fit
iso_forest.fit(df_merged[x_names[1:]])

In [ ]:
# predyckaj outlierów
is_outlier = iso_forest.predict(df_merged[x_names[1:]])

In [ ]:
# udział
pd.Series(is_outlier).value_counts()/df_merged.shape[0]

In [ ]:
# dodanie outlierow do danych
df_merged['outlier'] = is_outlier
df_wo_outliers = df_merged[df_merged['outlier']==1].reset_index(drop = True)

2. Optymalizacja modelu:
    - Wykorzystanie jednego z poznanych algorytmów optymalizacyjnych.
    - W przypadku niezadowalających wyników, testy na innym algorytmie.

In [ ]:
# minimalna data dla każdej stacji
min_date = df_wo_outliers.groupby('departure_id')['departure_date'].min().reset_index().rename(
    columns={'departure_date':'min_date'})
min_date['min_date'] = pd.to_datetime(min_date['min_date'])

In [ ]:
# stacje do odrzucenia 
stations_to_exclude = min_date[min_date['min_date']>'2019-12-31']['departure_id'].values

In [ ]:
# filtrowanie
df_wo_outliers = df_wo_outliers.merge(min_date, on ='departure_id')
df_wo_outliers = df_wo_outliers[~(df_wo_outliers['departure_id'].isin(stations_to_exclude))]
df_wo_outliers.shape

In [ ]:
# info
df_wo_outliers.info()

In [ ]:
# konwersja na daty
df_wo_outliers['departure_date'] = pd.to_datetime(df_wo_outliers['departure_date'] )

In [ ]:
# podzial na train / test / valid
train = df_wo_outliers[(df_wo_outliers['departure_date']> df_wo_outliers['min_date']) & (df_wo_outliers['departure_date']<='2019-12-31')]
test = df_wo_outliers[(df_wo_outliers['departure_date']>'2019-12-31') & (df_wo_outliers['departure_date']<='2020-06-30')]
valid = df_wo_outliers[(df_wo_outliers['departure_date']>'2020-07-01') ]
print(train.shape)
print(test.shape)
print(valid.shape)

In [ ]:
# podzial na x/y
train_x = train[x_names]
train_y = train['y_cat']
test_x = test[x_names]
test_y = test['y_cat']
valid_x = valid[x_names]
valid_y = valid['y_cat']


In [ ]:
import optuna
from sklearn.ensemble import HistGradientBoostingClassifier,RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
def objective(trial):
    model_name = trial.suggest_categorical('model_name',['adaboost','randomforest'])
    params = {'n_estimators':trial.suggest_int('n_estimators',1,200),
              'max_depth': trial.suggest_int('max_depth',1,10),
              'min_samples_split': trial.suggest_int('min_samples_split',10,100)}
    if model_name =='adaboost':
            params['max_iter'] = params['n_estimators']
            del params['n_estimators']
            model =HistGradientBoostingClassifier(**params).fit(train_x, train_y)
    else:
        model = RandomForestClassifier(**params).fit(train_x,train_y)
    preds = model.predict_proba(test_x)[:,1]
    return roc_auc_score(test_y,preds)


In [ ]:
study= optuna.create_study(direction='maximize')

In [ ]:
study.optimize(objective, n_trials=5,n_jobs=-1)

In [ ]:
# final model 
final_model = HistGradientBoostingClassifier(**study.best_params).fit(train_x,train_y)

In [ ]:
# predykcje
valid_pred = final_model.predict(valid_x)
valid_pred_proba = final_model.predict_proba(valid_x)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
# evaluation - classification report
print(classification_report(valid_y, valid_pred))

In [ ]:
# roc auc
roc_auc_score(valid_y,valid_pred_proba)

In [ ]:
import joblib

In [ ]:
# zapis modelu
joblib.dump(final_model, 'models/alert_model.joblib')